# Clustering d'Images avec Scikit-learn et Visualisation avec Streamlit

Le clustering est une technique d'apprentissage non supervisé qui permet de regrouper des données similaires en différents groupes ou clusters. Dans le contexte de l'analyse d'images, le clustering peut être utilisé pour segmenter des images, détecter des objets ou des régions d'intérêt, et comprendre la structure des données d'image. Ce TP vous guidera à travers le processus de clustering d'un ensemble d'images de chiffres manuscrits en utilisant l'algorithme K-Means et l'extraction de caractéristiques HOG et d'histogrammes de niveaux de gris. Vous visualiserez ensuite les résultats du clustering à l'aide de la bibliothèque Streamlit.


À la fin de ce TP, vous aurez une compréhension pratique du processus de clustering d'images, de l'extraction de caractéristiques, de l'évaluation des performances et de la visualisation des résultats à l'aide de Streamlit. Ces compétences sont précieuses dans de nombreux domaines tels que la vision par ordinateur, le traitement d'images médicales, l'analyse de données d'imagerie satellitaire, et bien d'autres.


In [7]:
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
import plotly.express as px
from features import *
from clustering import *
from utils import *
from constant import  PATH_OUTPUT, MODEL_CLUSTERING, PATH_DATA_ALL


## Partie 1 : Création du modèle de clustering d'images
#### (fichier pipeline.py)

**1. Chargement des données d'images de chiffres manuscrits à partir du dataset Digits.**
   - Vous utiliserez le célèbre dataset Digitis qui contient des images de chiffres manuscrits. Ce dataset est souvent utilisé pour tester des algorithmes de reconnaissance de chiffres et d'apprentissage automatique.



In [3]:
print("##### Chargement des données ######")

##### Chargement des données ######


In [8]:
# GET IMAGES
import os
from PIL import Image
import numpy as np

def load_images_from_folder(folder):
    images = []
    folder_numbers = []
    folder_names = []
    for folder_number, sub_folder in enumerate(sorted(os.listdir(folder))):
        sub_folder_path = os.path.join(folder, sub_folder)
        if os.path.isdir(sub_folder_path):
            folder_names.append(sub_folder)
            for filename in sorted(os.listdir(sub_folder_path)):
                img_path = os.path.join(sub_folder_path, filename)
                if os.path.isfile(img_path):
                    img = Image.open(img_path)
                    img_matrix = np.array(img)
                    images.append(img_matrix)
                    folder_numbers.append(folder_number)
    return images, folder_numbers, folder_names

# Example usage:
folder_path = PATH_DATA_ALL + "/code_test"
images, folder_numbers, folder_names = load_images_from_folder(folder_path)

# Now you have a list of images as matrices and a corresponding list of folder numbers.
# Each image in the 'images' list corresponds to the folder number in the 'folder_numbers' list.

**2. Extraction des caractéristiques HOG (Histogrammes de Gradients Orientés) et des histogrammes de niveaux de gris à partir des images.**
   - Les caractéristiques HOG capturent les informations de gradient et de bords dans les images, ce qui les rend utiles pour la reconnaissance d'objets et de formes.
   - Les histogrammes de niveaux de gris représentent la distribution des intensités de pixels dans l'image, fournissant des informations sur la texture et les motifs.

**TODO :**
   - Implémentez les fonctions `compute_hog_descriptors` et `compute_gray_histograms` dans  le fichier `features.py`, utilisez respectivement les fonctions `hog` de  la librairie `skimage` et  `calcHist` de `cv2`.
   - lien HOG : https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_hog.html
   - lien  histogrammes de niveaux de gris : https://pyimagesearch.com/2021/04/28/opencv-image-histograms-cv2-calchist/
   

In [ ]:

print("\n\n ##### Extraction de Features ######")
print("- calcul features hog...")
descriptors_hog = compute_hog_descriptors(images)
print("- calcul features Histogram...")
descriptors_hist = compute_gray_histograms(images)


**3. Application de l'algorithme K-Means sur les caractéristiques extraites pour obtenir les clusters.**
   - L'algorithme K-Means est un algorithme de clustering populaire qui partitionne les données en K clusters en minimisant la somme des carrés des distances entre les points de données et les centroïdes des clusters.
   
   
 
 **TODO :**
   - Dans le fichier `clustering.py` implémentez les fonctions `initialize_centers()`, `nearest_cluster()` et `fit()` du KMeans.
   

In [ ]:
print("\n\n ##### Clustering ######")
print(np.array(descriptors_hog)[0])
number_cluster = 10
kmeans_hog = KMeans(n_clusters=number_cluster)
kmeans_hist = KMeans(n_clusters=number_cluster)

print("- calcul kmeans avec features HOG ...")
kmeans_hog.fit(np.array(descriptors_hog))
print("- calcul kmeans avec features Histogram...")
kmeans_hist.fit(np.array(descriptors_hist))

**4. Évaluation des performances du clustering en utilisant des métriques**
- Score AMI (Adjusted Mutual Information) : 
    Le score AMI renvoie une valeur de 1 lorsque les deux partitions sont identiques (c'est-à-dire parfaitement appariées). Les partitions aléatoires (étiquetages indépendants) ont un score attendu autour de 0.
    

**5. Conversion des données de clustering au format requis pour la visualisation avec Streamlit.**

**TODO :**
- Dans le fichier `utils.py`, implémenter la fonction `conversion_3d` afin de convertir un vecteur de dimension n vers une dimension 3 pour a visualisation.
- Lien : https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

In [ ]:
list_dict = [metric_hist,metric_hog]
df_metric = pd.DataFrame(list_dict)

scaler = StandardScaler()
descriptors_hist_norm = scaler.fit_transform(descriptors_hist)
descriptors_hog_norm = scaler.fit_transform(descriptors_hog)

#conversion vers un format 3D pour la visualisation
x_3d_hist = conversion_3d(descriptors_hist_norm)
x_3d_hog = conversion_3d(descriptors_hog_norm)

# création des dataframe pour la sauvegarde des données pour la visualisation
df_hist = create_df_to_export(x_3d_hist, labels_true, kmeans_hist.labels_)
df_hog = create_df_to_export(x_3d_hog, labels_true, kmeans_hog.labels_)

# Vérifie si le dossier existe déjà
if not os.path.exists(PATH_OUTPUT):
    # Crée le dossier
    os.makedirs(PATH_OUTPUT)

# sauvegarde des données
df_hist.to_excel(PATH_OUTPUT+"/save_clustering_hist_kmeans.xlsx")
df_hog.to_excel(PATH_OUTPUT+"/save_clustering_hog_kmeans.xlsx")
df_metric.to_excel(PATH_OUTPUT+"/save_metric.xlsx")




In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
graph_size = 300


***3. Finalisation du fichier dashboard_clustering.py***

***TODO :***
- Ajoutez les graphiques dans le fichier, puis lancez la commande :  `streamlit run dashboard_clustering.py `